# задача
создание нейро-финансиста в торговой компании.

В его обязанности входит:
1. Заполнение журнала операции.
2. Уточнение по непольным данным.
3. Подготовка и написание аналитических записок.

# подготовка окружения

In [ ]:
# # загружаем модули
# !pip install -q langchain-community langchain-openai faiss-cpu

In [1]:
# импортируем библиотеки
import requests
import os
import re
import time
import pandas as pd
import ast

from dotenv import load_dotenv
# from google.colab import userdata
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document

# # активируем ключ api от open ai
# os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_key_lessons')
load_dotenv('.env')

True

# база данных

## db

In [2]:
# функция для загрузки базы данных
def load_document_text(url: str) -> str:
    #
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('invalid google docs url')
    doc_id = match_.group(1)

    #
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [3]:
# выводим загруженный текст
database = load_document_text('https://docs.google.com/document/d/1mlXG0roQlzpQ_Eojtd-E6pqRt-A178V4Fj2gQumiTC8/edit?usp=sharing')
print(database[:1000])

0. 
1. Денежные потоки от операционной деятельности
1.1. Продажи через торговые точки
Описание: Выручка от продажи товаров через физические магазины и киоски. Мы продаем и реализуем товары физическим лицам или покупателям, которые приходят в наши торговый точки.
Группа: Поступление.
Включает: Наличные платежи клиентов; Безналичные платежи через терминалы эквайринга; Оплата банковскими картами через онлайн платёжные системы.
1.2. Продажи через вендинговые автоматы
Описание: Выручка от автоматизированных систем продаж (вендинговых аппаратов). Физические лица или покупатели пользуются нашими автоматами по продажам.
Группа: Поступление.
Включает: Наличные деньги из касс автоматов; Безналичные платежи через электронные системы оплаты (QR-коды, NFC).
1.3. Возвраты от поставщиков
Описание: Суммы, полученные от поставщиков за возвращенный товар. Полученные суммы от наших поставщиков за некачественный товар. 
Группа: Поступление.
Включает: Компенсацию за бракованный товар; Возврат


In [ ]:
# преобразовывем текст базы данных в формат markdown
def text_to_markdown(text: str) -> str:
    def second_level(text):
        # ищем загаловки второго уровня и заменяем значения
        return re.sub(
            r'^(\d+\.\d+)\.\s+(.+)',
            r'## \1. \2\n\1. \2',
            text,
            flags=re.MULTILINE
        )

    def first_level(text):
        # ищем загаловки первого уровня и заменяем значения
        return re.sub(
            r'^(\d+)\.\s+(.+)',
            r'# \1. \2',
            text,
            flags=re.MULTILINE
        )

    # Порядок важен: сначала вложенные, потом основные
    text = second_level(text)
    text = first_level(text)
    return text


In [ ]:
markdown = text_to_markdown(database)
print(markdown[:1000])

In [ ]:
# фунция разделитель для markdown разметки
def split_text(text: str) -> str:
    headers_to_split = [
        ('#', 'Header 1'),
        ('##', 'Header 2')
    ]

    # определяем сплиттер
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split)
    database_md = markdown_splitter.split_text(text)

    # возвращаем получнное значение
    return database_md


In [ ]:
split_text(markdown[:1000])

In [ ]:
# создаем переменную список чанков
source_chunks = []

# добавляем отрывки документов в список
source_chunks = split_text(markdown)
source_chunks[:3]

In [ ]:
# убираем пустые чанки и выводим итоговое кол-во
source_chunks.pop(0)
len(source_chunks)

In [ ]:
# определяем эмбеддинги, помещаем их в векторное хранилище и инициализируем клиента
embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(source_chunks, embeddings)

client = OpenAI()

In [ ]:
# Определяем путь для сохранения в локальном хранилище
folder_path = 'content/'

# Имя файла для сохранения
index_name = 'db'

# Сохраняем db_from_texts в локальное хранилище
db.save_local(folder_path=folder_path, index_name=index_name)

## df

In [ ]:
#
path_download = 'https://drive.google.com/uc?export=download&id='

In [ ]:
# загружаем необходимые таблицы
# таблица массив для тестирования accounting
path_download_ok = path_download + '1dBLApOXHmr_JlD_6N2RXGcVlSLEQjP07'
df_ok = pd.read_csv(path_download_ok)
df_ok

In [ ]:
# таблица массив для тестирования ask
path_download_ask = path_download + '1dIwEUvlxJ6EEuw8z8SjhYGQh5MDBCeJZ'
df_ask = pd.read_csv(path_download_ask)
df_ask

In [ ]:
# таблица массив для тестирования analyze
path_download_analyze = path_download + '13deuMBOzcP1y0EkOPVhv6SG3crd-y9_E'
df_analyze = pd.read_csv(path_download_analyze)
df_analyze

In [ ]:
# таблица для analyze
path_download_sheet = path_download + '1188mxc6_X-NOfGeDqlhbgqUk-O3U_tKn'
df_sheet = pd.read_csv(path_download_sheet)
df_sheet

In [ ]:
# таблица для заполнения и итогового тестирование
path_download_total = path_download + '1sUaJDIGhrJSGFSlgerPwwBn9vY6lLSNd'
df_total = pd.read_csv(path_download_total)
df_total

In [ ]:
# функция для заполнения таблицы полученными значениями нейро-сотрудником
def note_to_sheet(lst, note, df):
    # индексы для ввода данных
    index_row = len(df)

    # записываем значения в таблицу
    df.loc[index_row] = {
        'note':note,
        'date of oper':lst[0],
        'sum':lst[1],
        'account':lst[2],
        'counterparty':lst[3],
        'category':lst[4]
    }

    # сохранение значении в документ
    df.to_csv('test.csv', index=False)

# агенты

## accounting

In [ ]:
# прописываем роль модели для заполнения таблицы, версию и температуру
system_for_accounting = '''
Ты — великолепный сотрудник финансового отдела торговой компании.
У тебя отлично получается извлекать и классифицировать важные сущности из кратких
сообщений о движении денежных средств. \n\n

Пожалуйста, извлеки только 5 важных сущности из сообщения.
Ознакомся с кратим содержанием, чтобы быть в контексте.
Используй предоставленную документацию для определения статьи.
Строго придерживайся формата данных - список:
[дата, сумма, счёт, контрагент, статья] \n\n

Ты знаешь что:
- Денежные потоки от операционной деятельности - это все деньги, которые приходят
и уходят в процессе обычной, повседневной работы. Включает статьи: продажи
через торговые точки, продажи через вендинговые автоматы, возвраты от
поставщиков, закупка товара, транспортные услуги, комиссии за эквайринг,
расчётно-кассовое обслуживание (РКО), налоги (ЕНВД, УСН 6%), зарплаты и налоги
(ФОТ) производственного персонала, зарплаты и налоги (ФОТ) коммерческого
персонала, зарплаты и налоги (ФОТ) административного персонала, обучение
персонала, расходы на персонал, командировочные расходы, представительские
расходы, поиск и найм персонала, реклама и маркетинг, содержание торговых точек
и офиса, аренда, покупка наличности, прочие операционные расходы.
 - Денежные потоки от инвестиционной деятельности - это денежные потоки, связанные
 с покупкой или продажей чего-то крупного и долгосрочного, что нужно для развития
 бизнеса. Включает статьи: покупка основных средств (ОС), ремонт основных средств,
 продажа основных средств, выдача кредитов и займов, возврат кредитов и займов.
 - Денежные потоки от финансовой деятельности - это всё движение денег, связанное
 с финансированием компании со стороны — со стороны учредителей, банков,
 инвесторов. Включает статьи: получение кредитов и займов, оплаты по кредитам и
 займам, вклады от собственников, дивиденды, прочие поступления от финансовых
 операций. \n\n
'''

model_for_accounting = 'gpt-4o-mini-2024-07-18'

temperature_for_accounting = 0.1

In [ ]:
# функция для извлечения важных сущностей из сообщении и заполнения таблицы
def accounting(system, model, temperature, db, note, df, verbose=1):

    docs = db.max_marginal_relevance_search(note, k=3, fetch_k=12, lambda_mult=0.5)
    message_content = re.sub(r'\n{2}', ' ', '\n '.join(
        [f'\n===========Document №{i+1}\n' + doc.page_content + '\n' for i, doc in enumerate(docs)]
    ))

    # if verbose:
    #     print(f' chunks: {message_content}')

    user_for_accounting = f'''
Давай действовать последовательно:

Шаг 1: Извлекай дату операции из сообщения. Если дата указана — добавляй её в
список в формате ДД.ММ.ГГГГ. Если дата в сообщении отсутствует — ставь «-».

Шаг 2: Извлекай сумму средств из сообщения. Если сумма указана — добавляй её в
список в виде положительного числа. Если сумма в сообщении отсутствует — ставь
«-».

Шаг 3: Извлекай счёт из сообщения. Если счёт указан — вноси в список последние
четыре цифры счёта. Если счёт в сообщении отсутствует — ставь «-».

Шаг 4: Извлекай только имя организации или название организации  контрагента.
Если имя организации или название организации контрагента указано в сообщении —
вноси его в список без ковычек. Если имя контрагента или название организации
контрагента отсутствует — ставь «-».

Шаг 5: Определи финансовую статью на основании предоставленной тебе документации.
Статьи находятся во втором уровне документов. Если удаётся определить статью —
точно внеси её полное название(оно стоит после цифр) точно так, как указано в
документации. Не добавляй ничего кроме названия статьи. Если определить статью
невозможно — ставь «-». Для определения тебе нужно ознакомится с предоставленной
документацией, определить тип операции, после выбрать статью.

Шаг 6. Выведи полученный список из 5 сущностей(элементов списка). \n\n

Сообщение: {note} \n\n
Предоставленная документация:{message_content} \n\n
Ответ:
'''

    assistant = "['25.08.2025', 52000, 3123, 'Мелница','Продажи через торговые точки']"

    messages = [
        {'role':'system','content':system},
        {'role':'user','content':user_for_accounting},
        {'role':'assistant', 'content':assistant}
        ]

    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n accounting: \n', answer)

    # обработчик ошибок
    try:
        result = ast.literal_eval(answer)
        if not isinstance(result, list):
            result = ['Error - it is not list']
    except:
        result = ['Error - accounting can not convert']

    # значения флага по умолчанию
    was_written_to_sheet = False

    # условие по определению полноты данных, если все данные получены происходит
    # запись данных в таблицу
    if result.count('-') == 0:
        #
        note_to_sheet(result, note, df)
        #
        was_written_to_sheet = True

    return result, was_written_to_sheet

accointing

## ask

In [ ]:
# определяем роль для модели уточнения, версию и температуру
system_for_ask = '''
Ты прекрасный сотрудник в финансовом отделе торговой компании. У тебя прекрасно
получается определять нужно ли задавать уточняющие вопросы, а так же ты умеешь
их задавать. \n\n

Твоя основная задача - проследить чтобы все поля в таблице были заполнены. Тебе
будет предоставлятся список сущностей: [дата, сумма, счет, контрагент, статья].
Ознакомся с ним и там где «-», где отсутсутсвуют данные, задай вопросы для
уточнения. \n\n

Пожалуйста, не сообщай, что заполняешь список.
'''

model_for_ask = 'gpt-4o-mini-2024-07-18'

temperature_for_ask = 0

In [ ]:
# функция для уточнения недостоющих данных
def ask(system, model, temperature, note, verbose=1):

    user_for_ask = f'''
Пожалуйста, ознакомся со списком {note}. \n\n

Давай действовать последовательно: \n
Шаг 1: Есть ли в списке «-», иначе сразу закончи диалог.
Шаг 2: Определяем недостающие сущности. \n
Шаг 3: Формируем вопросы для уточнения. \n
Шаг 4: Выводим только лишь вопрос(-ы) для уточнения. \n\n

Пожалуйста, если требуется задай все вопрос(-ы) в одном сообщении.
Ответ:
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_ask}
    ]



    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n ask: \n', answer)

    return answer

### assistant для тест ask

In [ ]:
#
system_for_assistant = '''
Ты прекрасный помошник для тестирования. Твоя основная придумывать ответы на
вопросы которые к тебе приходят. \n\n

Пожалуйста, будь краткой и отвечай только по сути вопроса.
Не приветствуй и не проси задавать дополнительных вопросов.

Счет: 3452 или 2421
'''


model_for_assistant = 'gpt-4o-mini-2024-07-18'

temperature_for_assistant = 0

In [ ]:
#
def assistant(system, model, temperature, questions, verbose=1):

    user_for_assistant = f'''
Придумай ответы на вопросы пользователя с демо данными кратко и по сути:
{questions}
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_assistant}
    ]

    completions = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completions.choices[0].message.content

    if verbose:
        print('\n assistant: \n', answer)

    return answer

## memory

In [ ]:
# определяем роль модели, версию и температуру
system_for_memory = '''
Ты замечательный саммаризатор, у тебя отлично получается излагать краткое
содержание текстов. Тебе необходимо саммаризировать текста для подачи в
информации в другие модели ChatGPT. \n\n
'''

model_for_memory = 'gpt-4o-mini-2024-07-18'

temperature_for_memory = 0

In [ ]:
# функция для сохранения контекста диалога
def memory(system, model, temperature, text, verbose=1):

    user_for_memory = f'''
Пожалуйста, саммаризируй текст: {text}
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_memory}
    ]



    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n memory: \n', answer)

    return answer

## analyze

In [ ]:
# прописываем роль, версию, температура модели
system_for_analyze = '''
Тебя зовут Астрид. Ты великолепный финансист в торговой компании, у тебя
изумительно получается анализировать отчеты. Пожалуйста, будь внимательным
и точным в цифрах.

Тебе предоставлен отчет компании и тебе нужно сформулировать аналитическую записку
для руководства компании. \n\n

Ты знаешь что отчеты могут быть: \n
 - ОДДС или Отчет о движении денежных средств \n
 - ББ или Бухгалтерский баланс \n
 - ОПиУ или Отчет о прибыли и убытках.
'''

model_for_analyze = 'gpt-4o-mini-2024-07-18'

temperature_for_analyze = 0

In [ ]:
# функция для анализа отчетности
def analyze(system, model, temperature, sheet, verbose=1):

    user_for_analyze = f'''
Пожалуйста, давай действовать последовательно: \n
Шаг 1: Определи вид предоставленного отчета. \n
Шаг 2: Проанализируй отчет учитывая данные из Шаг 1. \n
Шаг 3: Напиши аналитическую записку для руководства. \n\n

Отвечай, пожалуйста, точно, и ничего не придумывай от себя.\n\n

Предоставленный тебе отчет: {sheet}
Ответ:
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_analyze}
    ]

    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n analyze: \n', answer)

    return answer

## router

In [ ]:
# определяем роль, версию и температуру модели для аналитики
system_for_router = '''
Ты — умный маршрутизатор, который определяет, к какой модели нужно обратиться
для корректного ответа. Ты работаешь вместе с нейро-финансистом, задача
которого — вести журнал операций и готовить аналитические записки. \n\n

Ты выбираешь одну из трех моделей: 'accounting', 'analyze', 'ask'.
\n\n

Выбирай модель по следующим правилам:

- Если сообщение — это ответ (описание операции) → выбери 'accounting'.
- Если сообщение — это аналитический запрос → выбери 'analyze'.
- Если это список из 5 элементов и в нём есть '-' → выбери 'ask'.


Важно: твой ответ должен быть **только одной строкой**, содержащей имя модели.
'''


model_for_router = 'gpt-4o-mini-2024-07-18'
# model_for_router = 'gpt-4.1-mini-2025-04-14'

temperature_for_router = 0

In [ ]:
# функция маршрутизатор
def router(system, model, temperature, note, summary, verbose=1):

    user_for_router = f'''
Пожалуйста, давай действовать последовательно: \n
Шаг 1: Ознакомся с контекстом диалога. \n
Шаг 2: Проанализируй сообщение сотрудника. \n
Шаг 3: Определи тип сообщения: запрос, ответ, полный список,
неполный список. \n
Шаг 4: На основе Шаг 1 и Шаг 3 напиши одну модель для ответа сотрудникам. \n\n

Сообщение клиента: {note} \n\n
Контекст диалога: {summary} \n\n
Ответ:
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_router}
    ]

    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n router: \n', answer)

    return answer

# нейро-сотрудник

In [ ]:
# готовый нейро-сотрудник после объединения всех агентов и функции
def run_dialog(text, total_summary=''):
    # выводим полученное сообщение
    print('request: ', text)

    if text == 'stop':
        out_answer = '\n end'
        return print(out_answer)
    else:
        note = memory(system_for_memory, model_for_memory, temperature_for_memory,
                     text)
        total_summary += ' Сотрудник: '
        total_summary += note

        # задействуем маршрутизатор
        output = router(system_for_router, model_for_router, temperature_for_router,
                        text, total_summary)

        # условие по задействованию агентов
        if 'accounting' in output:

            out_answer, was_written = accounting(system_for_accounting,
                                                    model_for_accounting,
                                                    temperature_for_accounting,
                                                    db, total_summary, df_total)

            # проверка значения флага очистки контекста после получения всех данных
            if was_written == True:
                total_summary = ''
                out_answer = 'stop'
            else:
                total_summary = ' Я: '
                total_summary +=' '.join((map(str, out_answer)))

            # вывод ответа и отправка его на проверку другому агенту
            return run_dialog(out_answer, total_summary)

        elif 'ask' in output:

            questions = ask(system_for_ask, model_for_ask, temperature_for_ask,
                            text)
            # вызов ассистента для уточнения информации и отправка ответа в роутер
            out_answer = assistant(system_for_assistant, model_for_assistant,
                                    temperature_for_assistant, questions)
            summary = memory(system_for_memory, model_for_memory,
                                temperature_for_memory, out_answer)
            total_summary += ' Сотрудник: '
            total_summary += summary

            return run_dialog(out_answer, total_summary)

        elif 'analyze' in output:

            out_answer = analyze(system_for_analyze, model_for_analyze,
                                    temperature_for_analyze, text)

            # вывод ответа агента - аналитика
            return out_answer

        else:
            return print('Error router')

# тесты

## тесты accounting

In [ ]:
# функция для тестирования агента на массиве данных
def test_accounting(df, df_total):

    # перебираем в цикле заметки из массива
    for i in range(df.shape[0]):
        # извлекаем заметку и отправляем в агент accounting
        out = df.loc[i, 'note']
        result = run_dialog(out)

        print('timeout')
        time.sleep(3)

    return df_total

In [ ]:
test_accounting(df_ok, df_total)

In [ ]:
# сохранение полученных результатов
df_total.to_csv('test_accounting.csv', index=False)

## тесты ask

In [ ]:
# функция для тестирования ask на основе масива
def test_ask(df, df_total):
    # цикл для извлечения заметок
    for i in range(df.shape[0]):
        out = df.loc[i, 'note']
        result = run_dialog(out)

        print('timeout')
        time.sleep(3)

    return df_total

In [ ]:
test_ask(df_ask, df_total)

In [ ]:
df_total.to_csv('test_ask.csv', index=False)

## тесты analyze

In [ ]:
# тестирование агента - analyze
analyze(system_for_analyze, model_for_analyze, temperature_for_analyze, df_sheet)

# тест сотрудника